# Semi-Analytic Model - Parameter Selection

## Initialization

In [ ]:
import os
# import warnings
from pathlib import Path
import numpy as np
import scipy as sp
import scipy.optimize
import matplotlib.pyplot as plt
import matplotlib as mpl

import kalepy as kale

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
# warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})


In [ ]:
def func_line(xx, mm, bb):
        yy = mm * xx + bb
        return yy


def fit_values_linear(xx, yy, guess=[-1.0, 1.0]):
    popt, pcov = sp.optimize.curve_fit(func_line, xx, yy, p0=guess, maxfev=10000)
    return popt


def func_gauss(xx, aa, mm, ss):
    yy = aa * np.exp(-(xx - mm)**2 / (2.0 * ss**2))
    return yy


def fit_values_gaussian(xx, yy, guess=[1.0, 0.0, 1.0]):
    popt, pcov = sp.optimize.curve_fit(func_gauss, xx, yy, p0=guess, maxfev=10000)
    return popt

## Galaxy Stellar-Mass Function (GSMF)

### [Tomczak+2014](https://ui.adsabs.harvard.edu/abs/2014ApJ...783...85T/exportcitation)

- Includes single & double Schechter fits for various redshift bins, no fit to redshift evolution

In [ ]:
data = {
    0: {
        'z': [0.2, 0.5],
        'log10mstar': [11.05, 0.1],    # log10(M/Msol)
        'alpha': [-1.35, 0.04],
        'log10phistar': [-2.96, 0.10]  # log10(Phi / Mpc^-3 / dex)
    },
    1: {
        'z': [0.5, 0.75],
        'log10mstar': [11.00, 0.06],    # log10(M/Msol)
        'alpha': [-1.35, 0.04],
        'log10phistar': [-2.93, 0.07]  # log10(Phi / Mpc^-3 / dex)
    },
    2: {
        'z': [0.75, 1.00],
        'log10mstar': [11.16, 0.12],    # log10(M/Msol)
        'alpha': [-1.38, 0.04],
        'log10phistar': [-3.17, 0.11]  # log10(Phi / Mpc^-3 / dex)
    },
    3: {
        'z': [1.00, 1.25],
        'log10mstar': [11.09, 0.10],    # log10(M/Msol)
        'alpha': [-1.33, 0.05],
        'log10phistar': [-3.19, 0.11]  # log10(Phi / Mpc^-3 / dex)
    },
    4: {
        'z': [1.25, 1.50],
        'log10mstar': [10.88, 0.05],    # log10(M/Msol)
        'alpha': [-1.29, 0.05],
        'log10phistar': [-3.11, 0.08]  # log10(Phi / Mpc^-3 / dex)
    },
    5: {
        'z': [1.50, 2.00],
        'log10mstar': [10.97, 0.05],    # log10(M/Msol)
        'alpha': [-1.45, 0.05],
        'log10phistar': [-3.44, 0.08]  # log10(Phi / Mpc^-3 / dex)
    },
    6: {
        'z': [2.00, 2.50],
        'log10mstar': [11.28, 0.19],    # log10(M/Msol)
        'alpha': [-1.60, 0.08],
        'log10phistar': [-3.96, 0.19]  # log10(Phi / Mpc^-3 / dex)
    },
    7: {
        'z': [2.50, 3.00],
        'log10mstar': [11.35, 0.33],    # log10(M/Msol)
        'alpha': [-1.74, 0.12],
        'log10phistar': [-4.36, 0.29]  # log10(Phi / Mpc^-3 / dex)
    },    
}

In [ ]:
keys = list(data[0].keys())
print(f"{keys=}")
keys.pop(keys.index('z'))
fig, axes = plt.subplots(figsize=[10, 5], ncols=3)

nz = len(data)
redz = np.zeros((nz, 2))
values = np.zeros((3, nz, 2))

for ii, vals in enumerate(data.values()):
    zbin = vals['z']
    zave = np.mean(zbin)
    zwid = np.diff(zbin)/2.0
    redz[ii, 0] = zave
    redz[ii, 1] = zwid
    for jj, (key, ax) in enumerate(zip(keys, axes)):
        val = vals[key]
        values[jj, ii, :] = val[:]
        ax.errorbar(zave, val[0], xerr=zwid, yerr=val[1])
        if ii == 0:
            ax.set(xlabel='Redshift', title=key)
            ax.grid(True, alpha=0.25)

plt.show()

In [ ]:
keys = list(data[0].keys())
keys.pop(keys.index('z'))


fig, axes = plt.subplots(figsize=[10, 5], ncols=3)
nz = len(data)

for ax, vals in zip(axes, values):
    ax.set(xlabel='Redshift', title=key)
    ax.grid(True, alpha=0.25)
    xx = redz[:, 0]
    yy = vals[:, 0]
    ax.errorbar(xx, yy, xerr=redz[:, 1], yerr=vals[:, 1], ls='none')
    ax.plot(xx, yy, 'k--', alpha=0.5)

    fit = fit_values_linear(xx, yy)
    yy = xx * fit[0] + fit[1]
    print(fit)
    ax.plot(xx, yy, 'r--', alpha=0.5)

plt.show()

In [ ]:
NSAMPS = 1000
NSHOW = 20

keys = list(data[0].keys())
keys.pop(keys.index('z'))

fig, axes = plt.subplots(figsize=[10, 10], ncols=3, nrows=3)
plt.subplots_adjust(hspace=0.35)
nz = len(data)

fits = np.zeros((3, 2, NSAMPS))

for ii, (axrow, vals) in enumerate(zip(axes, values)):
    xx = redz[:, 0]
    # yy = vals[:, 0]
    yy = np.random.normal(vals[:, 0][:, np.newaxis], vals[:, 1][:, np.newaxis], size=(nz, NSAMPS))

    ax = axrow[0]
    ax.set(xlabel='Redshift', title=keys[ii])
    ax.grid(True, alpha=0.25)

    ax.errorbar(xx, vals[:, 0], xerr=redz[:, 1], yerr=vals[:, 1], ls='none')
    # ax.plot(xx, yy, 'k--', alpha=0.5)

    for jj in range(NSAMPS):
        fit = fit_values_linear(xx, yy[:, jj])
        fits[ii, 0, jj] = fit[0]
        fits[ii, 1, jj] = fit[1]

        if jj%(NSAMPS//NSHOW) == 0:
            # cc, = ax.plot(xx, yy[:, jj], alpha=0.5, lw=0.5)
            # cc.get_color()
            cc = None
            zz = xx * fit[0] + fit[1]
            ax.plot(xx, zz, ls='--', alpha=0.5, color=cc, lw=0.5)

    for jj, (ax, lab) in enumerate(zip(axrow[1:], ['slope', 'intercept'])):
        ax.set(xlabel=lab)
        ax.grid(True, alpha=0.25)
        kale.dist1d(fits[ii, jj], ax=ax, density=True)

plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=[10, 7], nrows=3, ncols=2)
plt.subplots_adjust(hspace=0.4)

labels = ['slope', 'intercept']

for (jj, ii), ax in np.ndenumerate(axes):
    ax.grid(True, alpha=0.25)
    ff = fits[jj, ii]
    kale.carpet(ff, ax=ax)
    
    xx = kale.utils.spacing(ff, scale='lin', num=100, stretch=0.0)
    xx, yy = kale.density(ff, points=xx, probability=True)
    ax.plot(xx, yy, 'k-', alpha=0.5)

    fit = fit_values_gaussian(xx, yy, [np.max(yy), np.mean(ff), np.std(ff)])
    yy = func_gauss(xx, *fit)
    ax.plot(xx, yy, 'r--', alpha=0.5)
    ax.set_title(f"{keys[jj]} - {labels[ii]} :: " + f"${fit[1]:.2f} \pm {fit[2]:.2f}$", fontsize=8)
    
plt.show()

### [Leja+2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...893..111L/abstract)

- Uses a double-Schechter function with parametrized redshift evolution

## Galaxy Pair Fraction (GPF)

### [Conselice+2003](https://ui.adsabs.harvard.edu/abs/2003AJ....126.1183C/abstract)

- No significant dependence on total mass, consistent with Duncan+2019

### [Gluck+2012](https://ui.adsabs.harvard.edu/abs/2012ApJ...747...34B/abstract)

- Strong mass-ratio dependence, but paramtrized in a very strange way with "mass range" (e.g. Eq.11), based on brightness ratio (??)

### [Mundy+2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.470.3507M/abstract)

- No significant dependence on total mass.
- Massive galaxies ($M_\star > 10^{11} M_\odot$): $f_0 = 0.025 \pm 0.004$, and $m=0.78\pm0.20$
- Intermediate galaxies ($M_\star > 10^{10} M_\odot$): $f_0 = 0.028 \pm 0.002$, and $m=0.80\pm0.09$

### [Duncan+2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...876..110D/abstract)

- Thorough examination of pair fractions vs. redshift, some discussion of mass ratio.
- Use Snyder+2017 merger timescale to convert between fractions and merger rates.
- Parametrize merger rate as a function of redshift as $f = f_0 \, \left(1+z\right)^m$
    - For mass bin $\log_{10}(M_\star) > 10.3$, they find $f_0 = 0.032^{+0.009}_{-0.007}$ and $m=0.844^{+0.216}_{-0.235}$
- Parametrize merger rate as a function of mass-ratio as $f_p(>\mu) = A \, \left(q^{-1} - 1\right)^B$
    - Fits vary with redshift, $\log_{10}A = \{-1.472, -1.522, -1.291, -1.299, -1.346\}$, and $B = \{0.413, 0.540, 0.515, 0.491, 0.582\}$

In [ ]:
bb = 0.5
qq = np.logspace(-4, 0, 100)
yy = np.power(1/qq - 1, bb)
plt.loglog(qq, yy)

zz = np.power(qq, -bb)
plt.loglog(qq, zz, 'k--')

zz = np.power((1-qq)/qq, bb)
plt.loglog(qq, zz, 'r--')

plt.show()

## Galaxy Merger Timescales (GMT)

### [Snyder+2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.468..207S/abstract)

- $\tau = 2.5 \left(1+z\right)^{-2} {\rm Gyr}$ leads to a good match between pairs and intrinsic merger rates

### [Conselice+2008](https://ui.adsabs.harvard.edu/abs/2008MNRAS.386..909C/abstract)

- Merger timescale with redshift dependence of $5.5\pm 2.5$ for highest mass bin, and more gradual for lower masses

### [Conselice+2009](https://ui.adsabs.harvard.edu/abs/2009MNRAS.399L..16C/abstract)

- Merger timescales with normalization between $0.6 \pm 0.3$ Gyr, and up to $1.1 \pm 0.3$ Gyr.

### [Boylan-Kolchin+2008](https://ui.adsabs.harvard.edu/abs/2008MNRAS.383...93B/abstract)

Theory & Simulations for fitting function of dynamical-friction time-scale.  End up with
$$\tau_{\rm merge} / \tau_{\rm dyn} =
    A \frac{q^{-b}}{\ln\left(1 + q^{-1}\right)} \left[ \frac{r_c(E)}{r_{\rm vir}} \right]^d \exp\left[c \cdot \eta\right]$$
finding: $A = 0.216$, $b=1.3$, $c=1.9$, and $d=1.0$.

Here $t_{\rm dyn} = 0.1 H(z)^{-1}$ is the dynamical time at the virial radius,
$q \equiv M_{\rm sat}/M_{\rm host}$ is the mass-ratio,
$\eta = j / j_c(E)$ is the orbital circularly (angular momentum relative to that of a circular orbit with the same energy),
and $r_c(E)$ is the radius of a circular orbit of the given energy.